## BRONZE TO SILVER LAYER


In [1]:
import pandas as pd
import os
import pathlib
from common_utilities import replace_punctuation_from_columns

In [2]:
## Folder and File path
cwd = pathlib.Path(os.getcwd())

bronze_data_path = cwd.joinpath("../DATA/BRONZE").resolve()

silver_path = cwd.joinpath("../DATA/SILVER")
silver_path.mkdir(parents=True, exist_ok=True)

### Bronze Layer - Trade History


In [3]:
# Initialize an empty list to store DataFrames
dfs = []

# List all CSV files in the folder
bronze_data_trade_history_file_path = bronze_data_path.joinpath("TRADE_HISTORY")
csv_files = bronze_data_trade_history_file_path.glob("*.csv")

# Loop through the CSV files
for file_path in csv_files:
    # Read the CSV file
    df_bronze = pd.read_csv(file_path)
    # Append the DataFrame to the list
    dfs.append(df_bronze)

# Concatenate all DataFrames into one
df_trade_history = pd.concat(dfs, ignore_index=True)

df_trade_history.columns = replace_punctuation_from_columns(df_trade_history.columns)
df_trade_history.dropna(how="all", axis=1, inplace=True)

# sort the dataframe by date
df_trade_history = df_trade_history.sort_values(by=["date", "trade_time", "company"])

# Save the result as a json file
silver_trade_history_file_path = silver_path.joinpath("TradeHistory.json")
df_trade_history.to_json(silver_trade_history_file_path, orient="records", indent=4)
df_trade_history.info()

print("Silver Layer JSON file for trade history successfully created at:")
print(silver_trade_history_file_path.resolve())

<class 'pandas.core.frame.DataFrame'>
Index: 46 entries, 17 to 40
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             46 non-null     object 
 1   company          46 non-null     object 
 2   amount           46 non-null     float64
 3   exchange         46 non-null     object 
 4   segment          46 non-null     object 
 5   scrip_code       46 non-null     object 
 6   instrument_type  46 non-null     object 
 7   strike_price     46 non-null     object 
 8   expiry           2 non-null      object 
 9   trade_num        42 non-null     float64
 10  trade_time       42 non-null     object 
 11  side             46 non-null     object 
 12  quantity         46 non-null     float64
 13  price            46 non-null     float64
dtypes: float64(4), object(10)
memory usage: 6.4+ KB
Silver Layer JSON file for trade history successfully created at:
C:\Users\prashant.tripathi\Code\Unilever\Upsto